### Libraries and datasets required

original dataset from  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
Images: 3000 (2000 Training, 1000 Validation) 

In [1]:
import os
import pandas as pd
import tfrecorder
import wandb 
print(wandb.__version__)
import tensorflow as tf
import time

0.10.4


In [2]:
os.system('wandb login 57ed58d54691b4024a67bc775e075cb86868cb41')
os.environ['WANDB_NOTEBOOK_NAME'] = 'data_prep.ipynb'

### GCS Fuse to be able to use os utilities on GCS without copying data

In [12]:
!gcsfuse --implicit-dirs tfx-example-image-classification /home/jupyter/gcs/
#!fusermount -u /home/jupyter/gcs/

Using mount point: /home/jupyter/gcs
Opening GCS connection...
Opening bucket...
Mounting file system...
File system has been successfully mounted.


### Collect all image URIs

In [4]:
RUN_NAME=time.strftime("set_up_data_%Y%m%d_%H%M%S")
run = wandb.init(project='cats-dogs-keras', job_type='data', name=RUN_NAME)

wandb: Currently logged in as: jared-doit (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [13]:
wandb.log({"cat_examples": wandb.Image("/home/jupyter/gcs/catsdogs/train/cats/cat.1.jpg", caption="Cat1")})
wandb.log({"cat_examples": wandb.Image("/home/jupyter/gcs/catsdogs/train/cats/cat.2.jpg", caption="Cat2")})
wandb.log({"dog_examples": wandb.Image("/home/jupyter/gcs/catsdogs/train/dogs/dog.1.jpg", caption="Dog1")})
wandb.log({"dog_examples": wandb.Image("/home/jupyter/gcs/catsdogs/train/dogs/dog.2.jpg", caption="Dog2")})

In [14]:
myDir='/home/jupyter/gcs/catsdogs/'
format='.jpg'
fileList = []
for root, dirs, files in os.walk(myDir, topdown=False):
        for name in files:
            if name.endswith(format):
                fullName = os.path.join(root, name)
                fileList.append(fullName)
                
fileList[:10]

['/home/jupyter/gcs/catsdogs/train/cats/cat.0.jpg',
 '/home/jupyter/gcs/catsdogs/train/cats/cat.1.jpg',
 '/home/jupyter/gcs/catsdogs/train/cats/cat.10.jpg',
 '/home/jupyter/gcs/catsdogs/train/cats/cat.100.jpg',
 '/home/jupyter/gcs/catsdogs/train/cats/cat.101.jpg',
 '/home/jupyter/gcs/catsdogs/train/cats/cat.102.jpg',
 '/home/jupyter/gcs/catsdogs/train/cats/cat.103.jpg',
 '/home/jupyter/gcs/catsdogs/train/cats/cat.104.jpg',
 '/home/jupyter/gcs/catsdogs/train/cats/cat.105.jpg',
 '/home/jupyter/gcs/catsdogs/train/cats/cat.106.jpg']

In [15]:
df=pd.DataFrame(fileList)
df.columns = ['image_uri']
df.head()

,image_uri
0,/home/jupyter/gcs/catsdogs/train/cats/cat.0.jpg
1,/home/jupyter/gcs/catsdogs/train/cats/cat.1.jpg
2,/home/jupyter/gcs/catsdogs/train/cats/cat.10.jpg
3,/home/jupyter/gcs/catsdogs/train/cats/cat.100.jpg
4,/home/jupyter/gcs/catsdogs/train/cats/cat.101.jpg


In [16]:
df.loc[df['image_uri'].str.contains('train'), 'split'] = 'TRAIN'
df.loc[df['image_uri'].str.contains('validation'), 'split'] = 'VALIDATION'
df.loc[df['image_uri'].str.contains('|'.join(['train/cats', 'validation/cats'])), 'label'] = 'cats'
df.loc[df['image_uri'].str.contains('|'.join(['train/dogs', 'validation/dogs'])), 'label'] = 'dogs'
df = df[['split', 'image_uri', 'label']]
df.head()

,split,image_uri,label
0,TRAIN,/home/jupyter/gcs/catsdogs/train/cats/cat.0.jpg,cats
1,TRAIN,/home/jupyter/gcs/catsdogs/train/cats/cat.1.jpg,cats
2,TRAIN,/home/jupyter/gcs/catsdogs/train/cats/cat.10.jpg,cats
3,TRAIN,/home/jupyter/gcs/catsdogs/train/cats/cat.100.jpg,cats
4,TRAIN,/home/jupyter/gcs/catsdogs/train/cats/cat.101.jpg,cats


In [17]:
dflocal=df.copy()
dfgcs=df.copy()
dfgcs['image_uri'] = dfgcs['image_uri'].str.replace('/home/jupyter/gcs','gs://tfx-example-image-classification')
dflocal.to_csv('../data/datalocal1.csv', index=False)
dfgcs.to_csv('../data/datagcs1.csv', index=False)

In [18]:
artifact = wandb.Artifact(name='training_images', type='dataset')
artifact.add_reference('gs://tfx-example-image-classification/catsdogs/train/')
run.log_artifact(artifact)
artifact = wandb.Artifact(name='validation_images', type='dataset')
artifact.add_reference('gs://tfx-example-image-classification/catsdogs/validation/')
run.log_artifact(artifact)
artifact = wandb.Artifact(name='image_uris_csv', type='dataset')
artifact.add_file('../data/datagcs.csv')
run.log_artifact(artifact)
run.finish()

wandb: Generating checksum for up to 10000 objects with prefix "catsdogs/train/"... Done. 0.2s
wandb: Generating checksum for up to 10000 objects with prefix "catsdogs/validation/"... Done. 0.1s


_step,3
_runtime,1588
_timestamp,1607635097


_step,▁▃▆█
_runtime,▁▅▆█
_timestamp,▁▅▆█


In [19]:
RUN_NAME=time.strftime("generate_tfrecords_%Y%m%d_%H%M%S")
run = wandb.init(project='cats-dogs-keras',job_type='data', name=RUN_NAME)
artifact = run.use_artifact('training_images:latest')
artifact = run.use_artifact('validation_images:latest')
artifact = run.use_artifact('image_uris_csv:latest')

wandb: wandb version 0.10.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [22]:
FILENAME='../data/datagcs.csv'
TFRECORD_OUTPUT='gs://tfx-example-image-classification/catsdogs/tfrecords'
PROJECT='jared-playground'
REGION='us-central1'
TFRECORDER_WHEEL='/home/jupyter/tfrecorder-2.0-py3-none-any.whl'


dfgcs = pd.read_csv(FILENAME)
dfgcs.tensorflow.to_tfr(
    output_dir=TFRECORD_OUTPUT,
    runner='DataflowRunner',
    project=PROJECT,
    region=REGION,
    tfrecorder_wheel=TFRECORDER_WHEEL)

{'job_id': '2020-12-10_13_28_32-1622342719623853266',
 'dataflow_url': 'https://console.cloud.google.com/dataflow/jobs/us-central1/2020-12-10_13_28_32-1622342719623853266?=project=jared-playground',
 'tfrecord_dir': 'gs://tfx-example-image-classification/catsdogs/tfrecords/tfrecorder-20201210-212812-to-tfr'}

In [23]:
dfgcs

,split,image_uri,label
0,TRAIN,gs://mchrestkha-demo-env-ml-examples/catsdogs/...,cats
1,TRAIN,gs://mchrestkha-demo-env-ml-examples/catsdogs/...,cats
2,TRAIN,gs://mchrestkha-demo-env-ml-examples/catsdogs/...,cats
3,TRAIN,gs://mchrestkha-demo-env-ml-examples/catsdogs/...,cats
4,TRAIN,gs://mchrestkha-demo-env-ml-examples/catsdogs/...,cats
...,...,...,...
2995,VALIDATION,gs://mchrestkha-demo-env-ml-examples/catsdogs/...,dogs
2996,VALIDATION,gs://mchrestkha-demo-env-ml-examples/catsdogs/...,dogs
2997,VALIDATION,gs://mchrestkha-demo-env-ml-examples/catsdogs/...,dogs
2998,VALIDATION,gs://mchrestkha-demo-env-ml-examples/catsdogs/...,dogs


In [24]:
artifact = wandb.Artifact(name='tfrecords', type='dataset')
artifact.add_reference('gs://tfx-example-image-classification/catsdogs/tfrecords/')
run.log_artifact(artifact)
run.finish()

AttributeError: 'Run' object has no attribute '_backend'